# モデル学習スクリプト

Hugging Faceモデルを学習するためのスクリプトです。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

## 1. 環境設定

まず、GPUが有効になっているか確認します。

**設定方法**: メニュー → ランタイム → ランタイムのタイプを変更 → GPU を選択

In [ ]:
# GPU確認
import torch
print(f"GPU利用可能: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU名: {torch.cuda.get_device_name(0)}")

## 2. ライブラリのインストール

In [ ]:
!pip install -q transformers datasets accelerate evaluate

## 3. ライブラリのインポート

In [ ]:
import argparse
import logging
import os
from typing import Any, Dict, Optional

import evaluate
import numpy as np
import torch
import yaml
from transformers import (
    AutoTokenizer,
    EarlyStoppingCallback,
    Trainer,
    TrainingArguments,
    set_seed,
)

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

## 4. 学習関数の定義

In [ ]:
def load_config(config_path: str) -> Dict[str, Any]:
    """
    設定ファイルを読み込む

    Args:
        config_path: 設定ファイルのパス

    Returns:
        設定辞書
    """
    with open(config_path, "r", encoding="utf-8") as f:
        config = yaml.safe_load(f)
    return config


def compute_metrics_classification(eval_pred):
    """
    分類タスクのメトリクスを計算

    Args:
        eval_pred: 予測結果

    Returns:
        メトリクス辞書
    """
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


def train(config: Dict[str, Any]):
    """
    モデルの学習を実行

    Args:
        config: 設定辞書
    """
    # シードの設定
    set_seed(config.get("seed", 42))

    # デバイスの設定
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Using device: {device}")

    # トークナイザーの読み込み
    logger.info(f"Loading tokenizer: {config['model']['name']}")
    tokenizer = AutoTokenizer.from_pretrained(config["model"]["name"])

    # データセットの準備（簡略化のため直接読み込み）
    from datasets import load_dataset
    dataset = load_dataset(config["dataset"]["name"])
    
    # 前処理
    def preprocess_function(examples):
        return tokenizer(
            examples['text'],
            padding="max_length",
            truncation=True,
            max_length=config["model"].get("max_length", 512),
        )
    
    processed_dataset = dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=['text']
    )

    # モデルの構築
    from transformers import AutoModelForSequenceClassification
    model = AutoModelForSequenceClassification.from_pretrained(
        config["model"]["name"],
        num_labels=config["model"]["num_labels"],
    )
    model.to(device)

    # 学習設定
    training_config = config["training"]
    training_args = TrainingArguments(
        output_dir=training_config["output_dir"],
        num_train_epochs=training_config["num_epochs"],
        per_device_train_batch_size=training_config["batch_size"],
        per_device_eval_batch_size=training_config["eval_batch_size"],
        learning_rate=training_config["learning_rate"],
        weight_decay=training_config.get("weight_decay", 0.01),
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_steps=training_config.get("logging_steps", 100),
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        save_total_limit=training_config.get("save_total_limit", 3),
        fp16=torch.cuda.is_available(),
        report_to="none",
        seed=config.get("seed", 42),
    )

    # Trainerの初期化
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=processed_dataset['train'],
        eval_dataset=processed_dataset.get('validation', processed_dataset.get('test')),
        compute_metrics=compute_metrics_classification,
    )

    # 学習の開始
    logger.info("Starting training...")
    train_result = trainer.train()

    # 学習結果の保存
    logger.info("Saving model...")
    trainer.save_model()
    tokenizer.save_pretrained(training_config["output_dir"])

    # メトリクスの保存
    metrics = train_result.metrics
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)

    # 評価
    if "test" in processed_dataset:
        logger.info("Evaluating on test set...")
        test_metrics = trainer.evaluate(processed_dataset["test"])
        trainer.log_metrics("test", test_metrics)
        trainer.save_metrics("test", test_metrics)

    logger.info("Training completed!")
    return train_result

## 5. 使用例

In [ ]:
# 学習設定
config = {
    "seed": 42,
    "model": {
        "name": "distilbert-base-uncased",
        "num_labels": 2,
        "max_length": 256
    },
    "dataset": {
        "name": "imdb"
    },
    "training": {
        "output_dir": "./results",
        "num_epochs": 1,  # デモ用に1エポック
        "batch_size": 16,
        "eval_batch_size": 32,
        "learning_rate": 2e-5,
        "weight_decay": 0.01,
        "logging_steps": 100,
        "save_total_limit": 2
    }
}

# 学習の実行
train_result = train(config)
print("学習完了!")
print(f"最終損失: {train_result.training_loss:.4f}")